______
<img style="float: right;" src="/files/tutorial/images/headerlogo.png">
## Google Earth Engine Tutorial
### Automatic data load
_____

In a previous notebook data were load to a GEE asset using the Javascript interface. In this section, a script will be used to upload several images contained in a folder. 

In [49]:
__author__ = 'yang'
__maintainer__ = 'Raul Zurita-Milla' , 'Rosa Aguilar'
__email__ = "r.zurita-milla@utwente.nl", "rosamaguilar@gmail.com"

# -------------------------------------------------------------------------------
# Name:        gee_upload_asset_v3loop_v6
# Purpose:     Load Images to assets in GEE
# 
# -------------------------------------------------------------------------------
import urllib
# import urllib2
from http.cookiejar import CookieJar
# Build opener
jar = cookielib.CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(jar))

#----------IMPORTS & INITS
import glob
import requests
from urllib import parse
import os
from os import path
import ee
from requests_toolbelt.multipart import encoder
import logging
import sys
import ast
from bs4 import BeautifulSoup

if sys.version_info > (3, 0):
    from urllib.parse import unquote
else:
    from urllib import unquote


os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
ee.Initialize()

ModuleNotFoundError: No module named 'cookielib'

In [ ]:
#--------------------Parameters-------------------------------------
username =  "a.torres@aggiemail.usu.edu"    # your gmail account
password =  "Faustino4652!"      # your password


# # ASSET folder or collection name and ASSET name:
# your asset id
# asset_dir = "/home/jovyan/work/notebooks/git-repos/prj_earthengine_hydroshare/asset_upload"
# your local directory
# img_dir =  "/home/jovyan/work/notebooks/git-repos/prj_earthengine_hydroshare/asset_upload"
# suffix for the file description that will be created for each image. This file will contain the assetid of the image in GEE 
name = "_desc.txt"   # suffix

In [ ]:
#---------- Define URLs---------------------------------------------------
google_accounts_url = 'https://accounts.google.com'
authentication_url = 'https://accounts.google.com/ServiceLoginAuth'
appspot_url = 'https://code.earthengine.google.com/assets/upload/geturl'

In [ ]:
def google_authenticate(username, password):
#     session = requests.session()

#     login_html = session.get(google_accounts_url)
#     soup_login = BeautifulSoup(login_html.content, 'html.parser').find('form').find_all('input')
#     payload = {}
#     for u in soup_login:
#         if u.has_attr('value'):
#             payload[u['name']] = u['value']

#     payload['Email'] = username
#     payload['Passwd'] = password

#     auto = login_html.headers.get('X-Auto-Login')
#     follow_up = parse.unquote(parse.unquote(auto)).split('continue=')[-1]
# #     galx = login_html.cookies['GALX']

#     payload['continue'] = follow_up
# #     payload['GALX'] = galx

#     session.post(authentication_url, data=payload)
#     return session
    '''
        Authenticates to Google Accounts using user-provided username and password    
    '''

    # Build up headers
    user_agent = 'Mozilla/5.0 (Ubuntu; X11; Linux i686; rv:8.0) Gecko/20100101 Firefox/8.0'
    headers = {'User-Agent' : user_agent}

    # Set Data to None
    data = None

    # Build up URL request with headers and data    
    request = urllib.request(google_accounts_url, data, headers)
    response = request_url(request)

    # Build up POST data for authentication
    html = response.read()
    dsh = BeautifulSoup(html).findAll(attrs={'name' : 'dsh'})[0].get('value').encode()

    auto = response.headers.getheader('X-Auto-Login')
    
    follow_up = urllib.unquote(urllib.unquote(auto)).split('continue=')[-1]

    galx = jar._cookies['accounts.google.com']['/']['GALX'].value

    values = {'continue' : follow_up,
              'followup' : follow_up,
              'dsh' : dsh,
              'GALX' : galx,
              'pstMsg' : 1,
              'dnConn' : 'https://accounts.youtube.com',
              'checkConnection' : '',
              'checkedDomains' : '',
              'timeStmp' : '',
              'secTok' : '',
              'Email' : username,
              'Passwd' : password,
              'signIn' : 'Sign in',
              'PersistentCookie' : 'yes',
              'rmShown' : 1}
    
    data = urllib.urlencode(values)
    
    # Build up URL for authentication
    session = urllib2.Request(authentication_url, data, headers)
    return session

In [ ]:
def get_uploadUrl(session):
    '''
    get the GEE asset upload url
    '''
#     _ = session.get('https://ee-api.appspot.com/assets/upload/geturl?')
#     _=session.get("https://code.earthengine.google.com/assets/upload/geturl")
    r=session.get("https://code.earthengine.google.com/assets/upload/geturl")

#     r = session.get('https://ee-api.appspot.com/assets/upload/geturl?')
#     if r.text.startswith('\n<!DOCTYPE html>'):
#         logging.error('Incorrect credentials. Probably. If you are sure the credentials are OK, refresh the authentication token. '
#                       'If it did not work report a problem. They might have changed something in the Matrix.')
#         sys.exit(1)
    d = ast.literal_eval(r.text)

    return d['url']

In [ ]:
def upload_file(session, file_path, asset_name, use_multipart, properties=None, nodata=None):

    with open(file_path, 'rb') as f:
        upload_url = get_uploadUrl(session)

        if use_multipart:
            form = encoder.MultipartEncoder({
                "documents": (file_path, f, "application/octet-stream"),
                "composite": "NONE",
            })
            headers = {"Prefer": "respond-async", "Content-Type": form.content_type}
            resp = session.post(upload_url, headers=headers, data=form)
        else:
            files = {'file': f}
            resp = session.post(upload_url, files=files)

        gsid = resp.json()[0]
        asset_data = {"id": asset_name,
                      "tilesets": [
                          {"sources": [
                              {"primaryPath": gsid,
                               "additionalPaths": []
                              }
                          ]}
                      ],
                      "bands": [],
                      "properties": properties,
                      "missingData": {"value": 255}
                      }
    return asset_data



#

In [ ]:
#----------Main Section
# username = ''
# password = ''

google = google_authenticate(username, password)
print (google)

In [ ]:
# # ASSET folder or collection name and ASSET name:
asset_dir = 'users/atorres/'
img_dir = '/home/jovyan/work/notebooks/git-repos/prj_earthengine_hydroshare/asset_upload/'
name = "_Multi.txt"   # suffix
os.chdir(img_dir)
files = glob.glob('*.tif')
# LOOP all TIF images in a given folder
for n in range(0,len(files)):
    im = files[n]
    asset_name = asset_dir + files[n][:-4]
    print ("asset", n, asset_name)
    #Local file path
    img = img_dir + im    #+'/'+ files[n]
    print (img)
    textFileName = img_dir + im[:-4] + name
    if os.path.isfile (textFileName):
        print (textFileName)
        print ('file already in GEE')

    else:
        print ('uploading file')
        asset_request = upload_file(google, img, asset_name, False)
        taskid = ee.data.newTaskId(1)[0]
        print ('ingesting...', taskid)
        ret = ee.data.startIngestion(taskid, asset_request)
        print ('ingesting...', ret)
        asset_request = 0

        with open(textFileName, "w") as text_file:
            text_file.write(str.format(asset_name))

In [ ]:
%tb